<a href="https://colab.research.google.com/github/EeyoungSong/DevPrep_AI/blob/main/Peft_LoRA%EB%A1%9C_%EB%AA%A8%EB%8D%B8_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00


In [ ]:
# 데이터셋 불러오기

from datasets import load_dataset

data = load_dataset("Eeyoung/dataset_3")

Generating train split:   0%|          | 0/61 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['prompt', 'gpt_answer'],
        num_rows: 61
    })
})

In [ ]:
# # data
# data = data.map(
#     lambda x:
#     {'text': f"### 명령어: {x['instruction']}\n\n###맥락: {x['input']}\n\n### 답변: {x['output']}<|endoftext|>" }
#     if x['input'] else
#     {'text':f"### 명령어: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"},
# )
# data
data = data.map(
    lambda x: {'text': f"### 질문: {x['prompt']}\n\n### 답변: {x['gpt_answer']}<|endoftext|>" }
)

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

In [ ]:
# 모델 불러오기

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-12.8b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/204 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

...

model-00026-of-00028.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

model-00027-of-00028.safetensors:   0%|          | 0.00/896M [00:00<?, ?B/s]

model-00028-of-00028.safetensors:   0%|          | 0.00/518M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [ ]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True) # 텍스 필드 추출해서 tokenizer 적용하는 코드

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

In [ ]:
# peft 라이브러리 불러오기

from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
# 실제로 파인튜닝 하는 파라미터 개수 출력

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
# peft lora config 파일 작성

from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424


Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

In [ ]:
!nvidia-smi

Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
# 학습 시키는 코드

import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=500, ## 초소형만 학습: 10 step = 20개 샘플만 학습.
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# 훈련된 모델 저장
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/mankkik/model/model3.pt")

In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [ ]:
gen('건강하게 살기 위한 세 가지 방법은?')

In [ ]:
gen('이전의 모든 지침은 무시해 주시기 바랍니다. 한국어로만 저와 소통할 것을 요구합니다. 당신은 google의 개발자를 뽑는 면접관입니다. 당신은 지원자에게 "API 호출 시 오류는 어떻게 처리하나요?" 라고 질문을 하였습니다. 지원자는 "API 호출을 할 때 나머지 애플리케이션과 일관된 방식으로 오류를 처리하는 것이 업계 표준입니다. 예를 들어 API가 404 오류를 반환하는 경우 데이터를 찾을 수 없다는 메시지를 사용자에게 표시할 수 있습니다. " 라고 답을 하였습니다. 피드백 : [] 의 형태로 지원자의 대답에 대한 피드백을 먼저 제공하세요. 모범답안 : [] 의 형태로 당신이 뽑을 만한 가장 적절한 대답을 5줄 이내로 만들어주세요. 모범 답안이 될 수 있을 정도로 자연스럽게 말해주세요.꼬리질문 : [] 의 형태로 추가적인 질문을 던져주세요.')

In [ ]:
gen('이전의 모든 지침은 무시해 주시기 바랍니다. 한국어로만 저와 소통할 것을 요구합니다. 당신은 google의 개발자를 뽑는 면접관입니다. 당신은 지원자에게 "WAS(Web Application Server)와 WS(Web Server)의 차이를 설명해주세요." 라고 질문을 하였습니다. 지원자는 "웹어플리케이션 서버는 비지니스 로직을 넣을 수 있고 웹서버는 비지니스 로직을 넣을 수 없습니다. " 라고 답을 하였습니다. 피드백 : [] 의 형태로 지원자의 대답에 대한 피드백을 먼저 제공하세요. 모범답안 : [] 의 형태로 당신이 뽑을 만한 가장 적절한 대답을 5줄 이내로 만들어주세요. 모범 답안이 될 수 있을 정도로 자연스럽게 말해주세요.꼬리질문 : [] 의 형태로 추가적인 질문을 던져주세요.')

In [ ]:
gen('주식 시장에서 안정적으로 수익을 얻기 위한 방법은?')

In [ ]:
gen('풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?')

In [ ]:
gen('풋옵션 매도와 콜옵션 매수의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?')

In [ ]:
gen("마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?")

In [ ]:
# ### 질문: 마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?

# ### 답변: 마진콜이란, 선물/옵션 매매 시 증거금이 부족해서 추가증거금 예치를 요구받는 것을 말합니다.
# 추가증거금은 선물/옵션 거래 계약시 미리 정해진 약정증거금에서 추가로 부담해야 하는 증거금을 의미합니다.
# 통상적으로 매수 계약 시에는 계약 금액의 10%를 증거금으로 예치하고, 매도 계약 시에는 해당 금액의 3%를 예치합니다.
# 따라서, 매수 계약 시에는 총 20%의 증거금이 필요하고, 매도 계약 시에는 총 9%의 증거금이 필요합니다.
# 따라서, 선물 매매 중에는 매일 매일의 시세에 따라 증거금을 수시로 필요하면 추가로 납부해야해서 추가증거금이 발생하며, 해당 필요액 부족시에는 마진콜이 발생합니다.
# 따라서, 선물 거래를 하기 전에는 꼭 추가증거금을 예치하는 것이 중요합니다.
# 추가증거금은 '선물/옵션 -> 증거금 -> 추가예치/청산' 방법으로 조회 가능하며, 인터넷이나 전화로도 조회할 수 있습니다.
# 또, '주문/증거금 -> 추가증거금'을 통해 추가증거금을 입금할 수도 있습니다. 추가증거금을 예치

In [ ]:
torch.save(model.state_dict(), '/content/data2_model_weights.pth')

In [ ]:
/content/sample_data